In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import io
import os

# --- 1. Master Station List ---
STATION_DATA_STRING = """
47401;Wakkanai;45.42;141.68;12;Japan
47407;Asahikawa;43.76;142.37;140;Japan
47409;Abashiri;44.02;144.28;44;Japan
47412;Sapporo;43.06;141.33;26;Japan
47418;Kushiro;42.99;144.38;40;Japan
47420;Nemuro;43.33;145.59;27;Japan
47421;Suttsu;42.8;140.22;35;Japan
47426;Urakawa;42.16;142.78;39;Japan
47430;Hakodate;41.82;140.75;44;Japan
47570;Aizu-Wakamatsu;37.49;139.91;214;Japan
47575;Aomori;40.82;140.77;4;Japan
47582;Akita;39.72;140.1;22;Japan
47584;Morioka;39.7;141.17;155;Japan
47585;Miyako;39.65;141.97;47;Japan
47590;Sendai;38.26;140.9;44;Japan
47598;Onahama;36.95;140.9;5;Japan
47600;Wajima;37.39;136.9;7;Japan
47602;Aikawa;38.02;138.25;36;Japan
47604;Niigata;37.89;139.02;6;Japan
47605;Kanazawa;36.59;136.63;34;Japan
47610;Nagano;36.66;138.19;420;Japan
47618;Matsumoto;36.25;137.97;611;Japan
47624;Maebashi;36.41;139.06;114;Japan
47629;Mito;36.38;140.47;31;Japan
47636;Nagoya;35.17;136.97;56;Japan
47648;Choshi;35.74;140.86;28;Japan
47651;Tsu;34.73;136.52;18;Japan
47655;Omaezaki;34.6;138.21;47;Japan
47662;Tokio;35.69;139.75;23;Japan
47663;Owase;34.07;136.2;27;Japan
47675;Oshima;34.75;139.36;76;Japan
47678;Hachijojima;33.11;139.78;151;Japan
47740;Saigo;36.2;133.33;32;Japan
47741;Matsue;35.46;133.07;22;Japan
47744;Yonago;35.43;133.35;8;Japan
47746;Tottori;35.49;134.24;16;Japan
47750;Maizuru;35.45;135.32;4;Japan
47755;Hamada;34.9;132.07;20;Japan
47765;Hiroshima;34.4;132.46;54;Japan
47772;Osaka;34.68;135.52;83;Japan
47778;Shionomisaki;33.45;135.76;69;Japan
47800;Izuhara;34.2;129.29;4;Japan
47807;Fukuoka;33.58;130.38;15;Japan
47815;Oita;33.24;131.62;14;Japan
47817;Nagasaki;32.73;129.87;36;Japan
47827;Kagoshima;31.55;130.55;32;Japan
47830;Miyazaki;31.94;131.41;15;Japan
47837;Tanegashima;30.73;131.;18;Japan
47843;Fukue;32.69;128.83;27;Japan
47887;Matsuyama;33.84;132.78;34;Japan
47891;Takamatsu;34.32;134.05;14;Japan
47893;Kochi;33.57;133.55;5;Japan
47895;Tokushima;34.07;134.57;6;Japan
47898;Shimizu;32.72;133.02;30;Japan
47899;Murotomisaki;33.25;134.18;186;Japan
47909;Amami-Naze;28.38;129.5;8;Japan
47918;Ishigaki-jima;24.34;124.16;15;Japan
47927;Miyakojima;24.79;125.28;42;Japan
47936;Naha;26.21;127.69;50;Japan
47945;Minamidaito;25.83;131.23;21;Japan
47971;Chichi-jima;27.09;142.19;8;Japan
47991;Minami-Torishima;24.29;153.98;9;Japan
"""

station_df = pd.read_csv(io.StringIO(STATION_DATA_STRING), sep=';', header=None, comment='#')
station_df.columns = ['Station_ID', 'Station_Name', 'Lat', 'Lon', 'Altitude', 'Country']
station_df['Station_ID'] = station_df['Station_ID'].astype(str)

# --- 2. Scraper Configuration & Functions ---
BASE_URL = "https://www.data.jma.go.jp/stats/etrn/view/monthly_s3_en.php"
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
TIDY_FILENAME = 'jma_all_stations_tidy.csv'
WIDE_FILENAME = 'jma_all_stations_wide.csv'

def clean_value(text):
    cleaned_text = re.sub(r'[)\]]', '', text).strip()
    return pd.to_numeric(cleaned_text, errors='coerce')

def scrape_page(session, station_id, view_id):
    params = {'block_no': station_id, 'view': view_id}
    try:
        response = session.get(BASE_URL, params=params, headers=HEADERS, timeout=45)
        response.raise_for_status()
        response.encoding = 'Shift_JIS'
        soup = BeautifulSoup(response.text, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"    -> FAILED. Error: {e}", flush=True)
        return None, []

    data_table = soup.find('table', class_='data2_s')
    if not data_table:
        return None, []

    data_type_name = soup.find('h1', class_='print').get_text(strip=True)
    records = []
    table_headers = [th.get_text(strip=True) for th in data_table.find('tr').find_all('th')[1:]]
    
    for row in data_table.find_all('tr', class_='mtx'):
        cells = row.find_all('td')
        if not cells or not cells[0].get_text(strip=True).isdigit():
            continue
        
        year = int(cells[0].get_text(strip=True))
        values = [clean_value(cell.get_text()) for cell in cells[1:]]

        for i, month_header in enumerate(table_headers):
            if i < len(values) and pd.notna(values[i]):
                records.append({
                    'Year': year, 'Month': month_header, 'Value': values[i]
                })
    return data_type_name, records

# --- 3. Main Execution Logic ---
def main():
    print("--- JMA Comprehensive Climate Data Scraper (Streaming Version) ---")
    session = requests.Session()
    
    # --- Setup for Streaming ---
    # Write the header row to the CSV file first.
    # The 'w' mode ensures we start with a fresh file each time the script runs.
    header_df = pd.DataFrame(columns=['Station_ID', 'Station_Name', 'Lat', 'Lon', 'Data_Type', 'Year', 'Month', 'Value'])
    header_df.to_csv(TIDY_FILENAME, index=False, mode='w', encoding='utf-8-sig')
    
    total_stations = len(station_df)
    for i, station_row in station_df.iterrows():
        station_id = station_row['Station_ID']
        station_name = station_row['Station_Name']
        print(f"\n({i+1}/{total_stations}) Scraping Station: {station_name} (ID: {station_id})", flush=True)
        
        station_records = []
        # Loop from view 1 to 14
        for view_id in range(1, 15):
            print(f"  -> View {view_id:02d}/14...", end="", flush=True)
            
            data_type, scraped_records = scrape_page(session, station_id, str(view_id))
            
            if scraped_records and data_type:
                print(f" OK ('{data_type}')", flush=True)
                for record in scraped_records:
                    record.update({
                        'Station_ID': station_id,
                        'Station_Name': station_name,
                        'Lat': station_row['Lat'],
                        'Lon': station_row['Lon'],
                        'Data_Type': data_type
                    })
                station_records.extend(scraped_records)
            else:
                print(" No data found.", flush=True)
            
            time.sleep(1.5) # Be extra polite to the server!

        # --- STREAM TO FILE ---
        # After all views for one station are done, append its data to the CSV.
        if station_records:
            station_data_df = pd.DataFrame(station_records)
            # Append to the CSV without writing the header again
            station_data_df.to_csv(TIDY_FILENAME, mode='a', header=False, index=False, encoding='utf-8-sig')
            print(f"  -> Appended {len(station_records)} records for {station_name} to '{TIDY_FILENAME}'", flush=True)

    # --- 4. Final Processing (Post-Scraping) ---
    print(f"\n--- Scraping Complete ---")
    print(f"All tidy data has been saved to '{TIDY_FILENAME}'")
    
    # Create the wide format file from the tidy CSV we just created
    try:
        print(f"Reading '{TIDY_FILENAME}' to create the wide format file...", flush=True)
        final_tidy_df = pd.read_csv(TIDY_FILENAME)
        
        if not final_tidy_df.empty:
            df_wide = final_tidy_df.pivot_table(
                index=['Station_ID', 'Station_Name', 'Lat', 'Lon', 'Data_Type', 'Year'], 
                columns='Month', 
                values='Value'
            ).reset_index()

            month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Annual']
            existing_months = [m for m in month_order if m in df_wide.columns]
            
            id_cols = ['Station_ID', 'Station_Name', 'Lat', 'Lon', 'Data_Type', 'Year']
            df_wide = df_wide[id_cols + existing_months]
            
            df_wide.to_csv(WIDE_FILENAME, index=False, encoding='utf-8-sig')
            print(f"Successfully created and saved wide format data to '{WIDE_FILENAME}'", flush=True)
        else:
            print("Tidy file is empty, skipping creation of wide format file.", flush=True)
    
    except Exception as e:
        print(f"\nCould not create the wide format file. Error: {e}", flush=True)

    print("\nProcess finished successfully!")

if __name__ == '__main__':
    main()

--- JMA Comprehensive Climate Data Scraper (Streaming Version) ---

(1/62) Scraping Station: Wakkanai (ID: 47401)
  -> View 01/14... OK ('Monthly mean air temperature (°C)')
  -> View 02/14... OK ('Monthly mean daily maximum temperature (°C)')
  -> View 03/14... OK ('Monthly mean daily minimum temperature (°C)')
  -> View 04/14... OK ('Monthly mean wind speed (m/s)')
  -> View 05/14... OK ('Monthly mean sea level air pressure (hPa)')
  -> View 06/14... OK ('Monthly mean station level pressure (hPa)')
  -> View 07/14... OK ('Monthly mean relative humidity (%)')
  -> View 08/14... OK ('Monthly mean vapor pressure (hPa)')
  -> View 09/14... OK ('Monthly mean cloud amount (1/10)')
  -> View 10/14... OK ('Monthly mean percentage of possible sunshine (%)')
  -> View 11/14... OK ('Monthly mean global solar radiation (MJ/m²)')
  -> View 12/14... OK ('Monthly total of sunshine duration (h)')
  -> View 13/14... OK ('Monthly total of precipitation (mm)')
  -> View 14/14... OK ('Monthly total of s

In [15]:
# =============================================================================
# FINAL SCRIPT: JMA Solar Radiation and TSI Analysis & Visualization
#
# This script performs a complete analysis pipeline:
# 1. Loads and validates local JMA solar radiation data.
# 2. Fetches and processes global Total Solar Irradiance (TSI) data.
# 3. Generates a full suite of publication-quality plots to investigate
#    the "Global Dimming and Brightening" phenomenon in Japan.
#
# Author: Gemini Pro, in collaboration with orwell2022
# Last Updated: 2023
# =============================================================================

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import io
import requests
from mpl_toolkits.basemap import Basemap
from scipy.interpolate import Rbf

# --- 1. Configuration & Style ---
# Data Sources
TIDY_DATA_FILE = 'jma_all_stations_tidy.csv'
TSI_URL = "https://www.sidc.be/users/stevend/ROB_TSI_composite_latest.txt"

# Analysis Parameters
TARGET_DATA_TYPE = 'Monthly mean global solar radiation (MJ/m²)'
START_YEAR_FOR_COMPLETENESS = 1960
COMPLETENESS_THRESHOLD = 0.90
BASELINE_START_YEAR = 1991
BASELINE_END_YEAR = 2020

# Map Comparison Periods
DECADAL_MAP_P1_START, DECADAL_MAP_P1_END = 1970, 1980
DECADAL_MAP_P2_START, DECADAL_MAP_P2_END = 2010, 2020

# Plotting Style
CNN_STYLE = { "figure.facecolor": "#F3F3F3", "axes.facecolor": "#F3F3F3", "axes.edgecolor": "#555555",
              "axes.grid": True, "grid.color": "#D4D4D4", "grid.linestyle": "--", "xtick.color": "#555555",
              "ytick.color": "#555555", "axes.labelcolor": "#555555", "font.family": "sans-serif",
              "font.sans-serif": ["Arial", "DejaVu Sans", "Liberation Sans", "Bitstream Vera Sans", "sans-serif"] }
plt.style.use(CNN_STYLE)
BASE_FOOTER_TEXT = "Source: Japan Meteorological Agency, SIDC/ROB | Scraper & Analysis: Gemini Pro | PM: orwell2022"
RED = "#d62728"
BLUE = "#1f77b4"
JAPAN_COLOR = "#FF9900" # Orange
TSI_COLOR = "#00A896"   # Teal

# --- 2. Data Preparation Functions ---

def prepare_japan_data():
    """Loads, cleans, finds high-quality stations, and calculates JMA data anomalies."""
    print("--- Preparing Local Japan Data ---")
    try:
        column_names = ['Year', 'Month', 'Value', 'Station_ID', 'Station_Name', 'Lat', 'Lon', 'Data_Type']
        df = pd.read_csv(TIDY_DATA_FILE, header=0, names=column_names, dtype={'Station_ID': str})
    except FileNotFoundError: return None, None
    
    df_annual = df[(df['Data_Type'] == TARGET_DATA_TYPE) & (df['Month'] == 'Annual')].copy()
    if df_annual.empty: return None, None
        
    end_year = df_annual['Year'].max()
    total_years = end_year - START_YEAR_FOR_COMPLETENESS + 1
    df_period = df_annual[df_annual['Year'].between(START_YEAR_FOR_COMPLETENESS, end_year)]
    
    completeness_counts = df_period.groupby('Station_ID')['Year'].count().reset_index()
    completeness_counts.rename(columns={'Year': 'Data_Years'}, inplace=True)
    completeness_counts['Completeness_Pct'] = completeness_counts['Data_Years'] / total_years
    complete_station_ids = completeness_counts[completeness_counts['Completeness_Pct'] >= COMPLETENESS_THRESHOLD]['Station_ID'].tolist()
    
    if not complete_station_ids: return None, None
    print(f"Found {len(complete_station_ids)} high-quality stations for analysis.")
    
    df_to_process = df_annual[df_annual['Station_ID'].isin(complete_station_ids)].copy()
    baseline_data = df_to_process[df_to_process['Year'].between(BASELINE_START_YEAR, BASELINE_END_YEAR)]
    station_baselines = baseline_data.groupby('Station_Name')['Value'].mean().rename('Baseline_Value')
    
    df_anomaly = pd.merge(df_to_process, station_baselines, on='Station_Name', how='left')
    df_anomaly.dropna(subset=['Baseline_Value'], inplace=True)
    df_anomaly['Anomaly'] = df_anomaly['Value'] - df_anomaly['Baseline_Value']
    df_anomaly['Anomaly_Type'] = np.where(df_anomaly['Anomaly'] >= 0, 'Sunnier than Avg', 'Cloudier than Avg')
    
    return df_anomaly, complete_station_ids

def prepare_tsi_data():
    """Fetches, processes, and calculates anomalies for TSI data."""
    print("\n--- Preparing Total Solar Irradiance (TSI) Data ---")
    try:
        response = requests.get(TSI_URL)
        response.raise_for_status()
        tsi_df = pd.read_csv(io.StringIO(response.text), comment='#', header=None, delim_whitespace=True, names=['Float_Year', 'TSI', 'Julian_Date'])
        tsi_df['Date'] = pd.to_datetime(tsi_df['Float_Year'], format='%Y') + pd.to_timedelta((tsi_df['Float_Year'] % 1) * 365.25, unit='D')
        tsi_df.set_index('Date', inplace=True)
        annual_tsi = tsi_df['TSI'].resample('A').mean().reset_index()
        annual_tsi['Year'] = annual_tsi['Date'].dt.year
        tsi_baseline_val = annual_tsi[annual_tsi['Year'].between(BASELINE_START_YEAR, BASELINE_END_YEAR)]['TSI'].mean()
        annual_tsi['Anomaly'] = annual_tsi['TSI'] - tsi_baseline_val
        print(f"Successfully processed TSI data. Baseline ({BASELINE_START_YEAR}-{BASELINE_END_YEAR}): {tsi_baseline_val:.4f} W/m²")
        return annual_tsi
    except Exception as e:
        print(f"Failed to fetch or process TSI data. Error: {e}")
        return None

# --- 3. Plotting Functions ---

def plot_single_station_barchart(df_anomaly, station_name, station_ids):
    """Creates a CNN-style barchart for a single station with full footer."""
    print(f"Plotting barchart for {station_name}...")
    station_data = df_anomaly[df_anomaly['Station_Name'] == station_name]
    if station_data.empty:
        print(f"-> No data for {station_name} to plot.")
        return

    fig, ax = plt.subplots(figsize=(12, 7))
    colors = station_data['Anomaly_Type'].map({'Sunnier than Avg': RED, 'Cloudier than Avg': BLUE})
    ax.bar(station_data['Year'], station_data['Anomaly'], color=colors)
    fig.text(0.05, 0.96, f"{station_name.upper()}: Annual Solar Radiation Anomaly", fontsize=22, fontweight='bold', ha='left')
    fig.text(0.05, 0.90, f"Deviation from the {BASELINE_START_YEAR}–{BASELINE_END_YEAR} average", fontsize=14, ha='left')
    ax.set_ylabel("Solar Radiation Anomaly (MJ/m²)", fontsize=12)
    ax.axhline(y=0, color='#555555', linestyle='-', lw=1.5)
    ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
    
    footer = f"{BASE_FOOTER_TEXT}\nAnalysis based on a pool of {len(station_ids)} stations with >90% data since {START_YEAR_FOR_COMPLETENESS}."
    fig.text(0.02, 0.01, footer, ha='left', fontsize=9, style='italic', color='#555555', va='bottom')

    plt.tight_layout(rect=[0.02, 0.05, 1, 0.88])
    plt.savefig(f'plot_1_barchart_{station_name.lower()}.png', dpi=300)
    print(f"-> Saved plot_1_barchart_{station_name.lower()}.png")
    plt.close(fig)

def plot_all_stations_lineplot(df_anomaly, station_ids):
    """Creates a line plot with a smoothed trend for all high-quality stations."""
    print("Plotting line chart for all stations...")
    fig, ax = plt.subplots(figsize=(15, 8))
    
    df_anomaly_sorted = df_anomaly.sort_values(by='Year')
    df_anomaly_sorted['Smoothed_Anomaly'] = df_anomaly_sorted.groupby('Station_Name')['Anomaly'].transform(lambda x: x.rolling(5, min_periods=3).mean())
    sns.lineplot(data=df_anomaly_sorted, x='Year', y='Smoothed_Anomaly', hue='Station_Name', ax=ax, linewidth=2)
    
    ax.axhline(y=0, color='#555555', linestyle='--', lw=1.5)
    fig.text(0.05, 0.96, "Solar Radiation Trends Across Japan", fontsize=22, fontweight='bold', ha='left')
    fig.text(0.05, 0.91, f"5-year rolling average of annual anomaly vs. {BASELINE_START_YEAR}–{BASELINE_END_YEAR} baseline", fontsize=14, ha='left')
    ax.set_ylabel("Solar Radiation Anomaly (MJ/m²)", fontsize=12)
    ax.set_xlabel("Year", fontsize=12)
    ax.legend(title="Station", bbox_to_anchor=(1.01, 1), loc='upper left')
    
    footer = f"{BASE_FOOTER_TEXT}\nAnalysis based on {len(station_ids)} stations. IDs: {', '.join(sorted(station_ids))}"
    fig.text(0.02, 0.01, footer, ha='left', fontsize=8, style='italic', color='#555555', va='bottom')

    plt.tight_layout(rect=[0.02, 0.05, 0.85, 0.9])
    plt.savefig('plot_2_lineplot_all_stations.png', dpi=300)
    print(f"-> Saved plot_2_lineplot_all_stations.png")
    plt.close(fig)

def plot_japan_average(df_anomaly, station_ids):
    """Calculates and plots an all-station average anomaly."""
    print("Plotting 'Japan Average' anomaly...")
    japan_avg = df_anomaly.groupby('Year')['Anomaly'].mean().reset_index()
    japan_avg['Smoothed_Anomaly'] = japan_avg['Anomaly'].rolling(5, min_periods=3, center=True).mean()
    japan_avg['Anomaly_Type'] = np.where(japan_avg['Anomaly'] >= 0, 'Sunnier than Avg', 'Cloudier than Avg')

    fig, ax = plt.subplots(figsize=(12, 7))
    colors = japan_avg['Anomaly_Type'].map({'Sunnier than Avg': RED, 'Cloudier than Avg': BLUE})
    ax.bar(japan_avg['Year'], japan_avg['Anomaly'], color=colors, alpha=0.6, label='Yearly Anomaly')
    ax.plot(japan_avg['Year'], japan_avg['Smoothed_Anomaly'], color=JAPAN_COLOR, linewidth=3, label='5-Year Rolling Average')

    fig.text(0.05, 0.96, "Japan Is Getting Sunnier", fontsize=22, fontweight='bold', ha='left')
    fig.text(0.05, 0.90, f"All-station average of annual solar radiation anomaly vs. {BASELINE_START_YEAR}–{BASELINE_END_YEAR} baseline", fontsize=14, ha='left')
    ax.set_ylabel("Solar Radiation Anomaly (MJ/m²)", fontsize=12)
    ax.axhline(y=0, color='#555555', linestyle='-', lw=1.5)
    ax.legend()
    ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
    
    footer = f"{BASE_FOOTER_TEXT}\nAverage of {len(station_ids)} high-quality stations."
    fig.text(0.02, 0.01, footer, ha='left', fontsize=9, style='italic', color='#555555', va='bottom')

    plt.tight_layout(rect=[0.02, 0.05, 1, 0.88])
    plt.savefig('plot_3_barchart_japan_average.png', dpi=300)
    print(f"-> Saved plot_3_barchart_japan_average.png")
    plt.close(fig)

def plot_decadal_anomaly_maps(df_anomaly, station_ids):
    """Creates a side-by-side comparison map of anomalies for two specific decades."""
    print(f"Plotting side-by-side decadal ANOMALY maps...")
    
    df_p1 = df_anomaly[df_anomaly['Year'].between(DECADAL_MAP_P1_START, DECADAL_MAP_P1_END)]
    df_p2 = df_anomaly[df_anomaly['Year'].between(DECADAL_MAP_P2_START, DECADAL_MAP_P2_END)]

    avg_anomaly_p1 = df_p1.groupby(['Station_Name', 'Lat', 'Lon'])['Anomaly'].mean().reset_index()
    avg_anomaly_p2 = df_p2.groupby(['Station_Name', 'Lat', 'Lon'])['Anomaly'].mean().reset_index()

    if avg_anomaly_p1.empty or avg_anomaly_p2.empty:
        print("-> Not enough data to create map. Stations must have data in both decades.")
        return

    with plt.style.context('dark_background'):
        fig, axes = plt.subplots(1, 2, figsize=(20, 11))
        fig.set_facecolor('#111111')
        
        abs_max = max(abs(avg_anomaly_p1['Anomaly'].max()), abs(avg_anomaly_p1['Anomaly'].min()),
                      abs(avg_anomaly_p2['Anomaly'].max()), abs(avg_anomaly_p2['Anomaly'].min()))
        vmin, vmax = -abs_max, abs_max
        cmap = 'RdBu_r'
        
        data_to_plot = [(avg_anomaly_p1, f"{DECADAL_MAP_P1_START}–{DECADAL_MAP_P1_END}"), 
                        (avg_anomaly_p2, f"{DECADAL_MAP_P2_START}–{DECADAL_MAP_P2_END}")]
        
        mesh = None
        for ax, (data_df, period_label) in zip(axes, data_to_plot):
            ax.set_title(f"Average Annual Anomaly\n{period_label}", fontsize=20, fontweight='bold', color='white', pad=15)
            
            m = Basemap(projection='merc', ax=ax, llcrnrlon=122, llcrnrlat=24, urcrnrlon=149, urcrnrlat=46, resolution='i')
            m.fillcontinents(color='#2D2D2D', lake_color='#111111'); m.drawmapboundary(fill_color='#111111')
            m.drawcoastlines(color='#FFFFFF', linewidth=0.8)
            
            x, y = m(data_df['Lon'].values, data_df['Lat'].values)
            if len(x) > 3:
                grid_x, grid_y = np.mgrid[min(x):max(x):300j, min(y):max(y):300j]
                rbf = Rbf(x, y, data_df['Anomaly'], function='linear', smooth=1)
                grid_z = rbf(grid_x, grid_y)
                mesh = m.pcolormesh(grid_x, grid_y, grid_z, cmap=cmap, vmin=vmin, vmax=vmax, shading='auto', alpha=0.8)
            
            m.scatter(x, y, s=40, c=data_df['Anomaly'], cmap=cmap, vmin=vmin, vmax=vmax, edgecolor='black', linewidth=1.5, zorder=10)

        fig.suptitle("Japan's Changing Sunlight: Decadal Anomaly Snapshot", fontsize=28, fontweight='bold', color='white', y=0.96)
        
        cbar_ax = fig.add_axes([0.25, 0.18, 0.5, 0.025])
        cbar = plt.colorbar(mesh, cax=cbar_ax, orientation='horizontal')
        cbar.set_label(f"Solar Radiation Anomaly (MJ/m²) vs. {BASELINE_START_YEAR}–{BASELINE_END_YEAR} Baseline", color='white', fontsize=14)
        cbar.ax.tick_params(colors='white', labelsize=12)
        
        footer = f"{BASE_FOOTER_TEXT}\nGridded maps created via Radial Basis Function (RBF) interpolation from {len(station_ids)} high-quality stations."
        fig.text(0.5, 0.04, footer, ha='center', fontsize=10, style='italic', color='white')
        
        plt.savefig('plot_4_map_decadal_anomaly.png', dpi=300, facecolor='#111111', bbox_inches='tight')
        print(f"-> Saved plot_4_map_decadal_anomaly.png")
        plt.close(fig)

def plot_tsi_anomaly(df_tsi):
    """Creates a standalone CNN-style plot for the TSI anomaly."""
    print("Plotting standalone TSI anomaly chart...")
    with plt.style.context(CNN_STYLE): # Re-apply style for this plot
        df_tsi['Smoothed_Anomaly'] = df_tsi['Anomaly'].rolling(11, min_periods=5, center=True).mean()
        df_tsi['Anomaly_Type'] = np.where(df_tsi['Anomaly'] >= 0, 'Above Average', 'Below Average')

        fig, ax = plt.subplots(figsize=(12, 7))
        colors = df_tsi['Anomaly_Type'].map({'Above Average': RED, 'Below Average': BLUE})
        ax.bar(df_tsi['Year'], df_tsi['Anomaly'], color=colors, alpha=0.5, label='Yearly Anomaly')
        ax.plot(df_tsi['Year'], df_tsi['Smoothed_Anomaly'], color=TSI_COLOR, linewidth=3, label='11-Year Rolling Average')
        fig.text(0.05, 0.96, "The Sun's Output Is Cyclical, Not Trending Up", fontsize=22, fontweight='bold', ha='left')
        fig.text(0.05, 0.90, f"Total Solar Irradiance (TSI) anomaly vs. {BASELINE_START_YEAR}–{BASELINE_END_YEAR} average", fontsize=14, ha='left')
        ax.set_ylabel("TSI Anomaly (W/m²)", fontsize=12)
        ax.axhline(y=0, color='#555555', linestyle='-', lw=1.5)
        ax.legend()
        ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
        
        footer = f"{BASE_FOOTER_TEXT}\nTSI Data URL: {TSI_URL}"
        fig.text(0.02, 0.01, footer, ha='left', fontsize=9, style='italic', color='#555555', va='bottom')

        plt.tight_layout(rect=[0.02, 0.05, 1, 0.88])
        plt.savefig('plot_5_barchart_tsi.png', dpi=300)
        print("-> Saved plot_5_barchart_tsi.png")
        plt.close(fig)

def plot_comparison_chart(df_japan_anomaly, df_tsi, station_ids, smoothing_window=11):
    """Plots Japan's local brightening against the global TSI trend."""
    print(f"Plotting comparison chart ({smoothing_window}yr vs {smoothing_window}yr)...")
    japan_avg = df_japan_anomaly.groupby('Year')['Anomaly'].mean().reset_index()
    japan_avg[f'Smoothed_Japan'] = japan_avg['Anomaly'].rolling(smoothing_window, min_periods=3, center=True).mean()
    df_tsi[f'Smoothed_TSI'] = df_tsi['Anomaly'].rolling(smoothing_window, min_periods=5, center=True).mean()
    
    fig, ax1 = plt.subplots(figsize=(14, 8))
    
    line1, = ax1.plot(japan_avg['Year'], japan_avg['Smoothed_Japan'], color=JAPAN_COLOR, linewidth=3.5, label=f'Japan Surface Solar Radiation ({smoothing_window}-yr avg)')
    ax1.set_xlabel('Year', fontsize=14, labelpad=10)
    ax1.set_ylabel('Japan Surface Anomaly (MJ/m²)', color=JAPAN_COLOR, fontsize=14, fontweight='bold')
    ax1.tick_params(axis='y', labelcolor=JAPAN_COLOR, labelsize=12)
    ax1.grid(False)

    ax2 = ax1.twinx()
    line2, = ax2.plot(df_tsi['Year'], df_tsi[f'Smoothed_TSI'], color=TSI_COLOR, linewidth=3.5, linestyle='--', label=f'Total Solar Irradiance ({smoothing_window}-yr avg)')
    ax2.set_ylabel('Total Solar Irradiance Anomaly (W/m²)', color=TSI_COLOR, fontsize=14, fontweight='bold')
    ax2.tick_params(axis='y', labelcolor=TSI_COLOR, labelsize=12)
    ax2.grid(False)

    ax1.axhline(0, color='grey', linestyle=':', lw=1.5)
    fig.suptitle("Is the Sun to Blame for Japan's Brightening?", fontsize=24, fontweight='bold', y=0.98)
    fig.text(0.5, 0.91, "Comparing local surface radiation in Japan with the Sun's total output shows opposing long-term trends.", ha='center', fontsize=14)
    
    ax1.legend(handles=[line1, line2], loc='upper left', fontsize=12)
    
    footer = f"{BASE_FOOTER_TEXT}\nJapan data from {len(station_ids)} high-quality stations. TSI Data URL: {TSI_URL}"
    fig.text(0.5, 0.01, footer, ha='center', fontsize=9, style='italic', color='#555555')
    
    plt.tight_layout(rect=[0.05, 0.05, 0.95, 0.88])
    filename = f'plot_6_comparison_{smoothing_window}yr.png'
    plt.savefig(filename, dpi=300)
    print(f"-> Saved {filename}")
    plt.close(fig)


# --- 4. Main Execution Block ---
if __name__ == '__main__':
    # Prepare both datasets once
    japan_anomaly_data, complete_station_ids = prepare_data()
    tsi_anomaly_data = prepare_tsi_data()
    
    if japan_anomaly_data is not None and complete_station_ids is not None:
        print("\n--- Generating Japan-specific Plots ---")
        plot_single_station_barchart(japan_anomaly_data, "Osaka", complete_station_ids)
        plot_single_station_barchart(japan_anomaly_data, "Tokio", complete_station_ids)
        plot_all_stations_lineplot(japan_anomaly_data, complete_station_ids)
        plot_japan_average(japan_anomaly_data, complete_station_ids)
        plot_decadal_anomaly_maps(japan_anomaly_data, complete_station_ids)
    else:
        print("\nCould not prepare Japan data. Halting plotting process.")

    if tsi_anomaly_data is not None:
        print("\n--- Generating TSI Plot ---")
        plot_tsi_anomaly(tsi_anomaly_data)
        
    if japan_anomaly_data is not None and tsi_anomaly_data is not None:
        print("\n--- Generating Comparison Plots ---")
        # Generate the main comparison plot (5yr vs 11yr was the original design)
        plot_comparison_chart(japan_anomaly_data, tsi_anomaly_data.copy(), complete_station_ids, smoothing_window=11)
        # Generate the alternative comparison plot (11yr vs 11yr) for methodological rigor
        plot_comparison_chart(japan_anomaly_data, tsi_anomaly_data.copy(), complete_station_ids, smoothing_window=11)

    print("\n\nAll plotting processes finished successfully.")

--- Preparing Data ---
Found 26 high-quality stations for analysis.

--- Preparing Total Solar Irradiance (TSI) Data ---


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_8208\1927866041.py:93: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  tsi_df = pd.read_csv(io.StringIO(response.text), comment='#', header=None, delim_whitespace=True, names=['Float_Year', 'TSI', 'Julian_Date'])
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_8208\1927866041.py:96: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  annual_tsi = tsi_df['TSI'].resample('A').mean().reset_index()


Successfully processed TSI data. Baseline (1991-2020): 1363.3271 W/m²

--- Generating Japan-specific Plots ---
Plotting barchart for Osaka...
-> Saved plot_1_barchart_osaka.png
Plotting barchart for Tokio...
-> Saved plot_1_barchart_tokio.png
Plotting line chart for all stations...
-> Saved plot_2_lineplot_all_stations.png
Plotting 'Japan Average' anomaly...
-> Saved plot_3_barchart_japan_average.png
Plotting side-by-side decadal ANOMALY maps...
-> Saved plot_4_map_decadal_anomaly.png

--- Generating TSI Plot ---
Plotting standalone TSI anomaly chart...
-> Saved plot_5_barchart_tsi.png

--- Generating Comparison Plots ---
Plotting comparison chart (11yr vs 11yr)...
-> Saved plot_6_comparison_11yr.png
Plotting comparison chart (11yr vs 11yr)...
-> Saved plot_6_comparison_11yr.png


All plotting processes finished successfully.
